In [ ]:
import matplotlib.pyplot as plt
import MDAnalysis as mda
import numpy as np
from matplotlib.markers import MarkerStyle as markerstyle
import math
from scipy.interpolate import griddata
import pandas
from scipy import constants
import matplotlib.patches as patches
import matplotlib.cm as cm

In [ ]:
#Energies_File
ener_file = "ADD PATH TO REWEIGHTED ENERGIES FILE HERE"  #Reweighted Energy file For e.g. - "Reweighted_Energies.dat"
dist_file = "ADD PATH TO MASTER DATA FILE"  # Distance file For e.g. - "Final_data.txt"

ener_data = pandas.read_csv(ener_file,comment='#',sep='\s+',dtype=np.float64)   #Reweighted Energy file
dist_data = pandas.read_csv(dist_file,comment='#',sep='\t',dtype=np.float64)   #Distance file
dist_data['Timestep'] = 200*round(dist_data['Timestep']/200)
#Remove duplicates 
dist_data = dist_data.drop_duplicates(subset=['Timestep'],ignore_index=True)
ener_data = ener_data.drop_duplicates(subset=['Timestep'],ignore_index=True)

# print("Energy Data: ", ener_data['Timestep'])

# print("Final_data: ", dist_data['Timestep'])

# # t_values = {'Timestep': dist_data['Timestep']}.values()
# mask1 = ener_data['Timestep'].isin(dist_data['Timestep'].values)
# print(mask1)
# ener_data = ener_data[~mask1]
# print(ener_data['Timestep'])



#Merge two dataframes
master_data = dist_data.join(ener_data.set_index('Timestep'),on='Timestep',how='inner',lsuffix = '_mda',rsuffix = '_plumed')

## Selecting bound and unbound configurations

In [ ]:


bound_mask = (master_data['d1_mda'] < 12.0) & (master_data['d2_mda'] < 12.0)
# bound_mask = (master_data['RMSD-B'] < 2.0)
unbound_mask = ((master_data['d1_mda'] > 12.0) | (master_data['d2_mda'] > 12.0)) & ((master_data['d1_mda'] < 21.0) & (master_data['d2_mda'] < 21.0))

bd_data = master_data[bound_mask]
unb_data = master_data[unbound_mask]

## Reweighting

Set rwt_bool = True to turn on reweighting

In [ ]:
rwt_bool = False
if rwt_bool:
    bd_rwt = bd_data['rbias']
    unb_rwt = unb_data['rbias']
else:
    bd_rwt = np.ones(len(bd_data))
    unb_rwt = np.ones(len(unb_data))

## Plotting parameters


In [ ]:
f_size=18
leg_size=16
tick_size=16
fig_size= (6,4)

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))

ax.plot(bd_data['d1_mda'],bd_data['d2_mda'],label='Bound',linewidth=1.2,linestyle='',color='steelblue',marker='o',markersize=5)
ax.plot(unb_data['d1_mda'],unb_data['d2_mda'],label='Unbound',linewidth=1.2,linestyle='',color='gold',marker='o',markersize=5)

ax.set_xlabel('d1 (nm)',fontsize=f_size)
ax.set_ylabel('d2 (nm)',fontsize=f_size)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
plt.show()

In [ ]:
ener_col = ['Bond','HP','G96','PDih','ImPDih','Col-Pro-Pro','LJ-Pro-Pro','Col-Pro-W','LJ-Pro-W','Col-W-W','LJ-W-W']
bound_energy = bd_data[ener_col].sum(axis=1)
unbound_energy = unb_data[ener_col].sum(axis=1)

bound_dict={}
unbound_dict={}

fig,ax = plt.subplots(figsize=fig_size)
bonded_terms = ['Bond','HP','G96','PDih','ImPDih']
bonded_sum_bd = bd_data[bonded_terms].sum(axis=1)
bonded_sum_unbd = unb_data[bonded_terms].sum(axis=1)
bound_dict['Bonded'] = [np.mean(bonded_sum_bd), np.std(bonded_sum_bd)]
unbound_dict['Bonded'] = [np.mean(bonded_sum_unbd),np.std(bonded_sum_unbd)]

ax.hist(bonded_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(bonded_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Bonded Interactions',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
fig.tight_layout()
plt.savefig('EnergyPlots/PS_Bonded.svg',format='svg',dpi=600,transparent=True)
plt.show()

In [ ]:
def plot_vs_cutoff(plot_data,ener_col,cutoffs,ax,colors=None):
    n_configs=[]
    prev_cutoff = 0
    for cutoff in cutoffs:
        
        mask = (plot_data['RMSD-B'] < cutoff) & (plot_data['RMSD-B'] >= prev_cutoff)
        data = plot_data[mask]
        energy = data[[ener_col]].sum(axis=1)
        if colors is not None:
            ax.hist(energy,bins=50,alpha=0.8,weights=np.ones(len(energy)),label=f'{cutoff} nm',density=True,color=colors[cutoff])
        else:
            ax.hist(energy,bins=50,alpha=0.8,weights=np.ones(len(energy)),label=f'{cutoff} nm',density=True)

        n_configs.append(len(energy))
        prev_cutoff = cutoff

    return n_configs

        


In [ ]:
fig,ax = plt.subplots(figsize=fig_size)
col_terms = ['Col-Pro-W']
colmb_sum_bd = bd_data[col_terms].sum(axis=1)
colmb_sum_unbd = unb_data[col_terms].sum(axis=1)
bound_dict[col_terms[0]] = [np.mean(colmb_sum_bd), np.std(colmb_sum_bd)]
unbound_dict[col_terms[0]] = [np.mean(colmb_sum_unbd),np.std(colmb_sum_unbd)]

ax.hist(colmb_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colmb_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Col (Pro-W) Energy Distribution',fontsize=f_size)
fig.tight_layout()

fig,ax = plt.subplots(figsize=fig_size)
col_terms = ['Col-Pro-Pro']
colmb_sum_bd = bd_data[col_terms].sum(axis=1)
colmb_sum_unbd = unb_data[col_terms].sum(axis=1)
bound_dict[col_terms[0]] = [np.mean(colmb_sum_bd), np.std(colmb_sum_bd)]
unbound_dict[col_terms[0]] = [np.mean(colmb_sum_unbd),np.std(colmb_sum_unbd)]

ax.hist(colmb_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colmb_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Col (Pro-Pro) Energy Distribution',fontsize=f_size)
fig.tight_layout()
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=fig_size)
lj_terms = ['LJ-Pro-W']
lj_sum_bd = bd_data[lj_terms].sum(axis=1)
lj_sum_unbd = unb_data[lj_terms].sum(axis=1)
bound_dict[lj_terms[0]] = [np.mean(lj_sum_bd), np.std(lj_sum_bd)]
unbound_dict[lj_terms[0]] = [np.mean(lj_sum_unbd),np.std(lj_sum_unbd)]

ax.hist(lj_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(lj_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('LJ (Pro-W) Energy Distribution',fontsize=f_size)
fig.tight_layout()

fig,ax = plt.subplots(figsize=fig_size)
lj_terms = ['LJ-Pro-Pro']
lj_sum_bd = bd_data[lj_terms].sum(axis=1)
lj_sum_unbd = unb_data[lj_terms].sum(axis=1)
bound_dict[lj_terms[0]] = [np.mean(lj_sum_bd), np.std(lj_sum_bd)]
unbound_dict[lj_terms[0]] = [np.mean(lj_sum_unbd),np.std(lj_sum_unbd)]

ax.hist(lj_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(lj_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('LJ (Pro-Pro) Energy Distribution',fontsize=f_size)
fig.tight_layout()
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
lj_terms='LJ-W-W'
ljW_bd= bd_data[lj_terms]
ljW_unbd = unb_data[lj_terms]
bound_dict[lj_terms] = [np.mean(ljW_bd), np.std(ljW_bd)]
unbound_dict[lj_terms] = [np.mean(ljW_unbd),np.std(ljW_unbd)]

ax.hist(ljW_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(ljW_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
# ax.hist(ljW_bd,bins=50,alpha=0.8,label='Bound',color='steelblue',density=True)
# ax.hist(ljW_unbd,bins=50,alpha=0.8,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_xticks([-1.085e7, -1.084e7,-1.083e7])
ax.ticklabel_format(axis='x',style='scientific',scilimits=(-1,3),useOffset=False)
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
# ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.set_title('LJ Energy (W-W) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/PS_LJ-WW.svg',format='svg',dpi=600,transparent=True)
plt.show()

fig,ax = plt.subplots(figsize=(6,4))
col_terms = 'Col-W-W'
colW_bd= bd_data[col_terms]
colW_unbd = unb_data[col_terms]
bound_dict[col_terms] = [np.mean(colW_bd), np.std(colW_bd)]
unbound_dict[col_terms] = [np.mean(colW_unbd),np.std(colW_unbd)]

ax.hist(colW_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colW_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.ticklabel_format(axis='x',style='scientific',scilimits=(-1,3),useOffset=False)
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Columb Energy (W-W) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/PS_Col-WW.svg',format='svg',dpi=600,transparent=True)
plt.show()